# Exercise 1: Agent Hello World

**Goal:** Create a simple AI agent with a custom weather tool using the OpenAI Agents SDK.

**Timebox:** 15 minutes

## What You'll Learn

- How to create an agent using the OpenAI Agents SDK
- How to define custom tool functions with `@function_tool`
- How to make real API calls (National Weather Service)
- How to run the agent with asyncio


> **💡 Tip:** If you get stuck, check the solution notebook in the `solutions/` directory! Each exercise has a corresponding complete solution that you can reference.
## Prerequisites

## Key Concepts: OpenAI Agents SDK

### What is the OpenAI Agents SDK?

The **OpenAI Agents SDK** is a modern framework for building agentic AI applications. It provides:

- **Simplified agent creation** with the `Agent` class
- **Automatic tool schema generation** with `@function_tool` decorator
- **Built-in conversation management** via `Runner`
- **Async/await support** for efficient I/O operations

### Architecture Pattern 🏗️

```
User Query 👤
    ↓
LLM interprets query and may choose tool 🤖
    ↓
[Tool chosen?]
    ↓ Yes
Tool Function: get_weather_alerts() 🔧
    ↓
External API Call (NWS) 🌐
    ↓
Data returned to Agent 📊
    ↓
Agent leverages LLM to generate response 💬
    ↓
Return to user ✅
```

This flow demonstrates how agents intelligently decide when to use tools to fulfill user requests!

### Tool Functions

Tool functions are Python functions that agents can call to perform actions:

- Decorated with `@function_tool` for automatic schema generation
- Can call external APIs, databases, or any Python code
- Return values are passed back to the LLM for final response generation
- The LLM decides **when** and **how** to use tools based on user queries

### Real API Integration

In this exercise, we use the **National Weather Service (NWS) API**:

- Provides free, real-time weather data for the United States
- No API key required (but User-Agent header recommended)
- Production-ready example of integrating external data sources
- Demonstrates how agents can access live information

### Why This Matters

This exercise teaches the **fundamental pattern** for all AI agents:
1. Define tools (functions the agent can use)
2. Create agent with instructions
3. Run agent with user query
4. Agent decides which tools to call
5. Tools return data
6. Agent generates final response

This same pattern scales from simple weather queries to complex multi-step workflows!

## Setup

Import required libraries and load environment variables.

Select **Python 3.11**** if prompted:

![Select Python 3.11 kernel](../../images/select-kernel.png)

In [ ]:
%pip install --quiet openai-agents httpx python-dotenv nest-asyncio

import os  # Operating system interface for environment variables
import asyncio  # Async I/O support for running async functions
import httpx  # Modern async HTTP client for making API requests
from dotenv import load_dotenv  # Load environment variables from .env file
from agents import Agent, Runner, function_tool  # OpenAI Agents SDK components
import nest_asyncio  # Allow nested async event loops in Jupyter notebooks

load_dotenv()  # Read .env file and load variables into os.environ

print("✅ Environment loaded")

## Step 1: Define a Weather Tool

Create a `get_weather_alerts()` function that calls the National Weather Service API.

**TODO:** Implement the function to fetch real weather alerts.

**Hints:**
- API endpoint: `https://api.weather.gov/alerts/active/area/{state}`
- Use `async with httpx.AsyncClient() as client`
- Set headers: `{"User-Agent": "OpenAI-Agents-Workshop (educational)"}`
- Parse JSON response and extract `features` array
- Handle cases where there are no active alerts

In [ ]:
@function_tool  # Decorator to register this function as an agent tool
async def get_weather_alerts(state: str) -> str:  # Async function that takes state code and returns alert description
    """
    Get current weather alerts for a US state from the National Weather Service.
    
    Args:
        state: Two-letter US state code (e.g., 'CA', 'NY', 'TX')
    
    Returns:
        String describing current weather alerts
    """
    print(f"🌐 Calling NWS API for state: {state}")

    try:
        # TODO fill in the url: https://api.weather.gov/alerts/active/area/
        url = f"/{state.upper()}"  # Construct API URL with uppercase state code
        headers = {"User-Agent": "OpenAI-Agents-Workshop (educational)"}  # Set User-Agent header (NWS API requirement)

        async with httpx.AsyncClient() as client:  # Create async HTTP client with automatic cleanup
            response = await client.get(url, headers=headers, timeout=10.0)  # Make GET request with 10s timeout
            response.raise_for_status()  # Raise exception if HTTP status indicates error

            data = response.json()  # Parse JSON response body
            features = data.get("features", [])  # Extract features array (contains alerts), default to empty list

            if not features:  # Check if no alerts found
                print("✅ No active alerts")
                return f"No active weather alerts for {state.upper()}."  # Return message indicating no alerts

            alerts = []  # Initialize list to store formatted alert strings
            for feature in features[:5]:  # Loop through first 5 alerts only
                properties = feature.get("properties", {})  # Get properties object from alert feature
                event = properties.get("event", "Unknown")  # Extract alert event type
                headline = properties.get("headline", "No headline")  # Extract alert headline text
                severity = properties.get("severity", "Unknown")  # Extract severity level
                alerts.append(f"- {event} ({severity}): {headline}")  # Format and add to alerts list

            result = f"Active weather alerts for {state.upper()}:\n" + "\n".join(alerts)  # Combine header with all formatted alerts
            print(f"✅ Found {len(features)} alert(s)")
            return result  # Return formatted alert string to agent

    except httpx.HTTPError as e:  # Catch HTTP-related errors (connection, timeout, 4xx/5xx status codes)
        error_msg = f"Failed to fetch weather alerts: {str(e)}"  # Create user-friendly error message
        print(f"❌ {error_msg}")
        return error_msg  # Return error to agent so it can inform user
    except Exception as e:  # Catch any other unexpected errors
        error_msg = f"Unexpected error: {str(e)}"  # Create generic error message
        print(f"❌ {error_msg}")
        return error_msg  # Return error to agent


print("✅ Weather tool defined with @function_tool")
print("ℹ️  This tool makes real API calls to weather.gov")

## Step 2: Create an Agent

Use the OpenAI Agents SDK to create an agent with instructions and tools.

**TODO:** Define the agent using the `Agent` class.

**Hints:**
- Give it a descriptive name (e.g., "Weather Agent")
- Write clear instructions explaining its purpose
- Pass your tool function in the `tools` list

In [ ]:
# TODO: Create an agent with the Agents SDK
# Hint: docs: https://openai.github.io/openai-agents-python/tools/#function-tools
# agent = Agent(
#     name="Apple Sorting Agent",
#     instructions="You help users get weather alert information for US states",
#TODO: you can pass in a tool or an apple, which will you choose?
#     tools=[]
# )

## Step 3: Verify OpenAI API Key

Ensure your OpenAI API key is configured.

In [ ]:
api_key = os.getenv("OPENAI_API_KEY")  # Retrieve API key from environment variables
if not api_key:  # Check if API key is missing or empty
    raise RuntimeError("OPENAI_API_KEY is not set. Add it to your .env file.")  # Stop execution with error message

masked_key = f"{api_key[:4]}...{api_key[-4:]}" if len(api_key) >= 8 else "***"  # Show first/last 4 chars for verification without exposing full key
print(f"✅ OPENAI_API_KEY detected: {masked_key}")

## Step 4: Run the Agent

Execute the agent with an example query.

**TODO:** Complete the `run_agent()` function to run the agent and display results.

In [ ]:
async def run_agent():  # Define async function to execute agent
    query = "Are there any weather alerts for California?"  # Define user query to send to agent
    print(f"Query: {query}\n")
    
    # TODO: Run the agent using Runner.run()
    # Hint: result = await Runner.run(agent, query)
    # TODO: Display the final output
    # Hint: print(f"\n🤖 Agent Response:\n{result.final_output}\n")
    
    print("⚠️  Implement the TODOs above!")
    print("   Check the solution notebook for reference")

# Jupyter-specific async handling (notebooks already have an event loop)
# In a regular Python script, you would use: asyncio.run(run_agent())
try:
    # Check if we're already in an async context (like Jupyter notebook)
    loop = asyncio.get_running_loop()
    # Allow nested async calls in the existing event loop
    nest_asyncio.apply()
    # Run our async function in the existing loop
    await run_agent()
except RuntimeError:
    # No event loop exists, so create a new one and run our function
    # This happens in regular Python scripts or non-async environments
    asyncio.run(run_agent())

## Expected Output

When you complete the TODOs, you should see output similar to:

```
Query: Are there any weather alerts for California?

🌐 Calling NWS API for state: CA
✅ Found 3 alert(s)

🤖 Agent Response:
Currently there are 3 active weather alerts for California including:
- Wind Advisory (Moderate): Wind Advisory until midnight
- High Surf Advisory (Moderate): High Surf Advisory until 9 PM
- Freeze Warning (Moderate): Freeze Warning from 2 AM to 9 AM

ℹ️  This used a real API call to the National Weather Service!
```

**What's happening:**
1. Your agent receives the query about California
2. The LLM decides to call `get_weather_alerts("CA")`
3. Your function makes a real API call to weather.gov
4. The API returns current alert data
5. The LLM formats the data into a natural language response

## Troubleshooting

### Error: `OPENAI_API_KEY is not set`

**Fix:** Ensure `.env` file exists in project root with your API key:
```
OPENAI_API_KEY=sk-your-key-here
```

### Error: Connection timeout or network error

**Possible causes:**
- Check your internet connection
- NWS API may be temporarily unavailable (normal behavior)
- Try a different state code
- Verify the API endpoint URL is correct

### No active alerts returned

**This is normal!** Many states have no active weather alerts at any given time.

**To test with active alerts:**
- Try different states: CA, TX, FL, NY, etc.
- Check [weather.gov](https://www.weather.gov) to find states with current alerts
- Winter months typically have more alerts

### Agent not calling the tool

**Possible causes:**
- Check that `tools=[get_weather_alerts]` is passed to `Agent()`
- Verify the function is decorated with `@function_tool`
- Ensure your query mentions weather or alerts for a US state

## Next Steps

Once you've completed this exercise, you'll understand:

- ✅ How to create agents with the OpenAI Agents SDK
- ✅ How to define custom tools that call real APIs
- ✅ How the agent decides when to use tools
- ✅ How to run agents with async/await

**Ready for more?** Proceed to:

- **[Exercise 2: Temporal Hello World](../02_temporal_hello_world/exercise.ipynb)** - Learn Temporal workflows and activities
- **[Exercise 3: Durable Agent](../03_durable_agent/exercise.ipynb)** - Combine agents + Temporal for production durability!